In [7]:
import psycopg2
import timeit

In [8]:
conn = psycopg2.connect("dbname='foodmining' user='penolove' host='localhost' password='password'")
cur = conn.cursor()
start = timeit.default_timer()
cur.execute("""select latlon,storename from storetable \
where (point(split_part(latlon, ',', 1)::numeric,split_part(latlon, ',', 2)::numeric)<@> point(25.0173405,121.53975179999998))<0.5""")
rows = cur.fetchall()
stop = timeit.default_timer()
print "calclute distance take :"+str(stop - start) +"s"
a=rows

#25.0173405,121.53975179999998
#24.7893351,120.9906655

calclute distance take :0.0378029346466s


# Solution2 : select for loop

In [9]:
r=[]
start = timeit.default_timer()
for i in a:
    cur.execute("select title,url from articletable where latlon='"+i[0]+"'")
    rows = cur.fetchall()
    r.append([i,(rows)])
stop = timeit.default_timer()
print "Find child takes :"+str(stop - start) +"s"
#25.0173405,121.53975179999998
#24.7893351,120.9906655

Find child takes :1.19280290604s


# Solution2 : select in 

In [10]:
start = timeit.default_timer()
start0 = timeit.default_timer()
tempstr=""
count=1
for i in a:
    if(count!=len(a)):
        tempstr+="'"+i[0]+"',"
    else:
        tempstr+="'"+i[0]+"'"
    count+=1
cur.execute("select title,url,latlon from articletable where latlon in ("+tempstr+")")
rows = cur.fetchall()
stop0 = timeit.default_timer()

start1= timeit.default_timer()
rx=dict()
for i in rows:
    if(rx.get(i[2],0)==0):
        rx[i[2]]=[i]
    else:
        rx[i[2]].append(i)
r1=rx.items()
stop1= timeit.default_timer()
stop= timeit.default_timer()

print "overall :"+str(stop - start) +"s"
print "Find child takes :"+str(stop0 - start0) +"s"
print "python operations :"+str(stop1 - start1) +"s"

# much faster

overall :0.139256000519s
Find child takes :0.138590097427s
python operations :0.000489950180054s


In [11]:
conn.close()